* The temporal order is scrambled in the test data making TS useless there.
* Still, just for learning / realism, we can still do it in the training data! 
* Let's add pseudo dates, and aggregate features on column subsets. Finally i'll run a model to predict the target!

In [1]:
import importlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [122]:
train = pd.read_csv("caltech-cs155-2020/train.csv")
test = pd.read_csv("caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,5,3,4,2,1,3,1,3,2,NaN
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,1,5,1,20,16,17,4,1,16,NaN
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,2,1,1,1,1,2,1,2,1,NaN
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,5,6,8,37,7,1,2,1,1,NaN
191858,784238,5200.0,5200.3,2.0,0.0,2.0,2,5200.0,5199.0,5198.8,...,5,1,2,1,1,1,4,1,3,NaN


In [123]:
train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [124]:
## y is binary.
display(train["y"].describe())

count    592380.000000
mean          0.356958
std           0.479103
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: y, dtype: float64

In [125]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

* Additional features could include: rank, which bid number is the max/min, etc' 
* features between the aggregated features (e.g. max bid div max ask..)

In [126]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
#     df[f"{group}_logsumexp"] = df[grou/_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [169]:
# df[bid_cols] = df[bid_cols] - df["mid"]



# for col in bid_cols:
#     df[f"{col}_sub"] = (df[col] - df["last_price"])/df["last_price"]
# for col in ask_cols:
#     df[f"{col}_sub"] = (df[col] - df["last_price"])/df["last_price"]
# for i in range(5):
#     df[f"{i}_imbal"] = (df[bid_vol_cols[i]]-df[ask_vol_cols[i]])/(df[bid_vol_cols[i]]+df[ask_vol_cols[i]])

# df["mid_sub"] = (df["mid"] - df["last_price"])/df["last_price"]

# df["interest_imbal"] = df["d_open_interest"]/df["transacted_qty"]
# df["opened_rat"] = df["opened_position_qty "]/df["transacted_qty"]
# df["closed_rat"] = df["closed_position_qty"]/df["transacted_qty"]

# df = df.drop(columns=bid_cols)
# df = df.drop(columns=ask_cols)



import feature_processing

df = feature_processing.preprocess(df)



# df[ask_cols] = df[ask_cols] - df["mid"]

KeyError: 'bid1'

In [142]:
print(df.columns)
df.head()

Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y', 'bid_cols_max',
       'bid_cols_min', 'bid_cols_spread', 'bid_vol_cols_max',
       'bid_vol_cols_min', 'bid_vol_cols_spread', 'ask_cols_max',
       'ask_cols_min', 'ask_cols_spread', 'ask_vol_cols_max',
       'ask_vol_cols_min', 'ask_vol_cols_spread', 'last_price_div__mid',
       'bid1_sub', 'bid2_sub', 'bid3_sub', 'bid4_sub', 'bid5_sub', 'ask1_sub',
       'ask2_sub', 'ask3_sub', 'ask4_sub', 'ask5_sub', '0_imbal', '1_imbal',
       '2_imbal', '3_imbal', '4_imbal', 'mid_sub', 'interest_imbal',
       'opened_rat', 'closed_rat'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,ask5_sub,0_imbal,1_imbal,2_imbal,3_imbal,4_imbal,mid_sub,interest_imbal,opened_rat,closed_rat
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,0.000416,0.142857,0.000000,0.714286,0.166667,0.500000,0.000052,0.000000,NaN,NaN
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,0.000520,0.750000,0.200000,0.466667,0.000000,-0.368421,0.000156,-0.781818,0.109091,0.890909
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,0.000468,0.500000,-0.882353,-0.428571,0.680000,0.142857,0.000078,-0.821429,0.083333,0.916667
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,0.000260,0.666667,0.300000,0.846154,0.000000,-0.466667,-0.000104,-0.810811,0.081081,0.918919
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,0.000468,0.866667,0.600000,0.333333,-0.692308,-0.578947,-0.000026,-0.853659,0.073171,0.926829


In [143]:
# df = df.drop(columns=bid_cols)
# df = df.drop(columns=ask_cols)

In [ ]:
df.head()

In [146]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

count                         784239
unique                        784239
top       2019-01-02 14:21:54.500000
freq                               1
first            2019-01-01 00:00:00
last             2019-01-05 12:55:19
Name: date, dtype: object

# Split back into train and test, and build model

# Define functions

In [ ]:
def get_normalized_X_and_train_y(df, col):
    train_opq = df.loc[~df[col].isna()]
    print(f"train shape {train.shape[0]}")
    test_opq = df.loc[df[col].isna()]
    print(f"test shape {test.shape[0]}")
    print(train_opq.shape, test_opq.shape)

    X_opq = train_opq.drop(["opened_position_qty ", "closed_position_qty", "d_open_interest","id","date","y"],axis=1)
    from scipy import stats
    normalized_X_opq = pd.DataFrame(stats.zscore(X_opq, axis=0))
    normalized_X_opq.columns = [X_opq.columns]
    y_opq = train_opq[col]

    print(normalized_X_opq.shape, y_opq.shape)


    test_X_opq = test_opq.drop(["opened_position_qty ", "closed_position_qty", "d_open_interest", "id","date","y"],axis=1)

    normalized_test_X_opq = pd.DataFrame(stats.zscore(test_X_opq, axis=0))
    normalized_test_X_opq.columns = [test_X_opq.columns]
    normalized_test_X_opq.index = test_X_opq.index
    print(normalized_test_X_opq.shape)
#     print(normalized_test_X_opq.columns)
    return normalized_X_opq, y_opq, normalized_test_X_opq, train_opq, test_opq

In [ ]:
def get_df_with_preds_for_nans(df, col):
    from sklearn import linear_model
    clf = linear_model.SGDClassifier(max_iter=10, tol=1e-3)
    clf.fit(normalized_X_opq, y_opq)

    pred_opq = clf.predict(normalized_test_X_opq)
    test_opq[col] = pred_opq
    df = pd.concat([train_opq,test_opq],sort=False)
    df = df.sort_values('id')
    return df

In [ ]:
# df

In [95]:
normalized_X_opq, y_opq, normalized_test_X_opq, train_opq, test_opq = get_normalized_X_and_train_y(df, "opened_position_qty ")

train shape 592380
test shape 191859
(784239, 48) (0, 48)
(784239, 42) (784239,)
(0, 42)


C:\Users\George\Anaconda3\lib\site-packages\scipy\stats\stats.py:2275: RuntimeWarning: Mean of empty slice.
  mns = a.mean(axis=axis)
C:\Users\George\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\George\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\George\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\George\Anaconda3\lib\site-packages\numpy\core\_methods.py:130: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [96]:
df = get_df_with_preds_for_nans(df, "opened_position_qty ")

C:\Users\George\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


ValueError: Found array with 0 sample(s) (shape=(0, 42)) while a minimum of 1 is required.

In [ ]:
df

In [ ]:
normalized_X_opq, y_opq, normalized_test_X_opq, train_opq, test_opq = get_normalized_X_and_train_y(df, "closed_position_qty")

In [ ]:
df = get_df_with_preds_for_nans(df, "closed_position_qty")

In [147]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

train shape 592380
test shape 191859


In [170]:
# train_no_nan = train.fillna(train.mean())
# test_no_nan = test.fillna(test.mean())
train_no_nan = train.fillna(-1)
test_no_nan = test.fillna(-1)

In [171]:
X = train_no_nan.drop(["id","date","y"],axis=1)
y = train_no_nan["y"]

In [163]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

grad_tree = GradientBoostingClassifier(n_estimators=110, learning_rate=0.01, max_leaf_nodes=8)


learning_rate = [0.01,0.01]
n_estimators = [110,110]
max_leaf_nodes = [8,8]

param_grid = dict(learning_rate=learning_rate,
                  n_estimators=n_estimators,
                  max_leaf_nodes=max_leaf_nodes)

In [164]:

from sklearn.model_selection import RandomizedSearchCV
random = RandomizedSearchCV(estimator=grad_tree,
                            param_distributions=param_grid,
                            scoring='roc_auc',
                            verbose=10, n_jobs=3,
                            n_iter=1)

In [165]:
random_result = random.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  9.5min remaining:    0.0s
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  9.5min finished


In [157]:
from get_average_val_err import get_val_err

get_val_err(3,train_no_nan,grad_tree)

592380
Fold  1  of  3  test indices: [     0      1      2 ... 197457 197458 197459]
len(val_index_list):  197460


C:\Users\George\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


validation error:  0.5171501115891574
Fold  2  of  3  test indices: [197460 197461 197462 ... 394917 394918 394919]
len(val_index_list):  197460


C:\Users\George\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


validation error:  0.5163122754483764
Fold  3  of  3  test indices: [394920 394921 394922 ... 592377 592378 592379]
len(val_index_list):  197460


C:\Users\George\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [183]:
from sklearn.ensemble import RandomForestClassifier
import randomforestorder
importlib.reload(randomforestorder)
model = randomforestorder.RandomForestOrder()
# model.fit(X,y)

In [ ]:
from get_average_val_err import get_val_err

get_val_err(3,train_no_nan,model)

592380
Fold  1  of  3  test indices: [     0      1      2 ... 197457 197458 197459]
len(val_index_list):  197460


C:\Users\George\Desktop\CS155_kaggle\actual\TeamAPlus\randomforestorder.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X,y)


validation error:  0.6632811590603351
Fold  2  of  3  test indices: [197460 197461 197462 ... 394917 394918 394919]
len(val_index_list):  197460


C:\Users\George\Desktop\CS155_kaggle\actual\TeamAPlus\randomforestorder.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X,y)


validation error:  0.6703751206291836
Fold  3  of  3  test indices: [394920 394921 394922 ... 592377 592378 592379]
len(val_index_list):  197460


C:\Users\George\Desktop\CS155_kaggle\actual\TeamAPlus\randomforestorder.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X,y)


In [152]:
model.fit(X, y)

In [168]:
import pickle
pickle.dump( model, open( "random_forest_model.p", "wb" ) )

In [153]:
y_prob = model.predict(test_no_nan.drop(["id","date","y"],axis=1))

In [155]:
print(y_prob)

[0.48200871 0.23207374 0.31251156 ... 0.35489187 0.34200657 0.32121727]


In [ ]:
model.predict_proba(test_no_nan.drop(["id","date","y"],axis=1))

## export predictions

In [154]:
# test["Predicted"] = model.predict(test.drop(["id","date","y"],axis=1),prediction_type='Probability')[:,1]
# test["Predicted"] = random_result.predict_proba(test_no_nan.drop(["id","date","y"],axis=1))[:,1]
test["Predicted"] = grad_tree.predict_proba(test_no_nan.drop(["id","date","y"],axis=1))[:,1]
# test["Predicted"] = y_prob
test[["id","Predicted"]].to_csv("submission_forest_feature_engineering_4.csv",index=False)

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
